# Data aggregations
With the dataset we built for T2, there is still some data processing we need to do:
- Manually tag ads (as found [here](https://docs.google.com/spreadsheets/d/1YuU6g32nyDuk7oKvLV-U1rbYdVdM3fKGIxJ3mT2-8vs/edit#gid=0))
- Incorporate ads and images descriptions to the dataset (NLP)

In [1]:
import pandas as pd
import fasttext
from sklearn import preprocessing

In [2]:
t2_df = pd.read_csv('../../data/AllUsers_Ads_Ratings_df.csv')
t2_df.head()

/var/folders/12/tpf_lyk108743n_1zhjxp3dw0000gn/T/ipykernel_29551/3869638846.py:1: DtypeWarning: Columns (22,28,29,30,36) have mixed types. Specify dtype option on import or set low_memory=False.
  t2_df = pd.read_csv('../../data/AllUsers_Ads_Ratings_df.csv')


,UserId,AdId,Age,Cap/Zip-Code,Countries visited,Fave Sports,Gender,Home country,Home town,Income,...,fave8,fave9,unfave1,unfave2,unfave3,unfave4,unfave5,unfave6,AdFilePath,Rating
0,U0001,A01_01,62,15613,United States of America,I do not like Sports,F,United States of America,Apollo,1,...,NaN,NaN,news headlines,homelessness,violence,war,human rights,NaN,../../data/ads16-dataset/ADS16_Benchmark_part1...,1.0
1,U0001,A01_02,62,15613,United States of America,I do not like Sports,F,United States of America,Apollo,1,...,NaN,NaN,news headlines,homelessness,violence,war,human rights,NaN,../../data/ads16-dataset/ADS16_Benchmark_part1...,1.0
2,U0001,A01_03,62,15613,United States of America,I do not like Sports,F,United States of America,Apollo,1,...,NaN,NaN,news headlines,homelessness,violence,war,human rights,NaN,../../data/ads16-dataset/ADS16_Benchmark_part1...,1.0
3,U0001,A01_04,62,15613,United States of America,I do not like Sports,F,United States of America,Apollo,1,...,NaN,NaN,news headlines,homelessness,violence,war,human rights,NaN,../../data/ads16-dataset/ADS16_Benchmark_part1...,1.0
4,U0001,A01_05,62,15613,United States of America,I do not like Sports,F,United States of America,Apollo,1,...,NaN,NaN,news headlines,homelessness,violence,war,human rights,NaN,../../data/ads16-dataset/ADS16_Benchmark_part1...,1.0


In [3]:
t2_df.columns

Index(['UserId', 'AdId', 'Age', 'Cap/Zip-Code', 'Countries visited',
       'Fave Sports', 'Gender', 'Home country', 'Home town', 'Income',
       'Last Name', 'Most listened musics', 'Most read books',
       'Most visited websites', 'Most watched movies',
       'Most watched tv programmes', 'Name', 'Paypal', 'Timepass',
       'Type of Job', 'Weekly working hours', 'fave1', 'fave10', 'fave2',
       'fave3', 'fave4', 'fave5', 'fave6', 'fave7', 'fave8', 'fave9',
       'unfave1', 'unfave2', 'unfave3', 'unfave4', 'unfave5', 'unfave6',
       'AdFilePath', 'Rating'],
      dtype='object')

## Favorite and unfavorite picture descriptions

In [4]:
t2_df['faves'] = t2_df[['fave1', 'fave2','fave3', 'fave4','fave5', 'fave6', 'fave7', 'fave8', 'fave9', 'fave10']].apply(lambda x: ' '.join(x.map(str)), axis = 1)
t2_df['unfaves'] = t2_df[['unfave1', 'unfave2','unfave3', 'unfave4','unfave5', 'unfave6']].apply(lambda x: ' '.join(x.map(str)), axis = 1)

In [5]:
t2_df.drop(columns = [
    'fave1', 
    'fave2',
    'fave3', 
    'fave4',
    'fave5', 
    'fave6', 
    'fave7', 
    'fave8', 
    'fave9', 
    'fave10',
    'unfave1', 
    'unfave2',
    'unfave3',
    'unfave4',
    'unfave5',
    'unfave6'
    ],
    axis = 1,
    inplace = True
)

In [6]:
sep = ' nan'
t2_df['faves'] = t2_df['faves'].apply(lambda x: x.split(sep, 1)[0])
t2_df['unfaves'] = t2_df['unfaves'].apply(lambda x: x.split(sep, 1)[0])

In [7]:
t2_df['faves']

0        my cats my cats movie we are in tv show we are...
1        my cats my cats movie we are in tv show we are...
2        my cats my cats movie we are in tv show we are...
3        my cats my cats movie we are in tv show we are...
4        my cats my cats movie we are in tv show we are...
                               ...                        
36115    nike Muhammad Ali House of Cards Peace Hidden ...
36116    nike Muhammad Ali House of Cards Peace Hidden ...
36117    nike Muhammad Ali House of Cards Peace Hidden ...
36118    nike Muhammad Ali House of Cards Peace Hidden ...
36119    nike Muhammad Ali House of Cards Peace Hidden ...
Name: faves, Length: 36120, dtype: object

### Embedding

In [8]:
def embedding_model(df_col, name):
    with open('../../data/' + name + '.txt', 'w') as f:
        for line in df_col.values:
            f.write(line)
            f.write('\n')
    
    model = fasttext.train_unsupervised('../../data/' + name + '.txt')

    t2_df[name + '_embeddings'] = df_col.apply(lambda x: model.get_sentence_vector(x))
    print(t2_df[name + '_embeddings'])

In [9]:
embedding_model(t2_df['faves'], 'faves')

Read 0M words
Number of words:  1432
Number of labels: 0
Progress: 100.0% words/sec/thread:   89983 lr:  0.000000 avg.loss:  0.348175 ETA:   0h 0m 0s


0        [0.018792206, -0.10480994, 0.055124957, -0.072...
1        [0.018792206, -0.10480994, 0.055124957, -0.072...
2        [0.018792206, -0.10480994, 0.055124957, -0.072...
3        [0.018792206, -0.10480994, 0.055124957, -0.072...
4        [0.018792206, -0.10480994, 0.055124957, -0.072...
                               ...                        
36115    [-0.007818802, -0.17089917, -0.012248972, -0.0...
36116    [-0.007818802, -0.17089917, -0.012248972, -0.0...
36117    [-0.007818802, -0.17089917, -0.012248972, -0.0...
36118    [-0.007818802, -0.17089917, -0.012248972, -0.0...
36119    [-0.007818802, -0.17089917, -0.012248972, -0.0...
Name: faves_embeddings, Length: 36120, dtype: object


In [10]:
embedding_model(t2_df['unfaves'], 'unfaves')

Read 0M words
Number of words:  1155
Number of labels: 0
Progress: 100.0% words/sec/thread:  103792 lr:  0.000000 avg.loss:  0.401065 ETA:   0h 0m 0s


0        [-0.03176648, -0.07896182, -0.09802474, 0.0389...
1        [-0.03176648, -0.07896182, -0.09802474, 0.0389...
2        [-0.03176648, -0.07896182, -0.09802474, 0.0389...
3        [-0.03176648, -0.07896182, -0.09802474, 0.0389...
4        [-0.03176648, -0.07896182, -0.09802474, 0.0389...
                               ...                        
36115    [-0.06939603, -0.104585275, -0.100376114, 0.18...
36116    [-0.06939603, -0.104585275, -0.100376114, 0.18...
36117    [-0.06939603, -0.104585275, -0.100376114, 0.18...
36118    [-0.06939603, -0.104585275, -0.100376114, 0.18...
36119    [-0.06939603, -0.104585275, -0.100376114, 0.18...
Name: unfaves_embeddings, Length: 36120, dtype: object


In [11]:
t2_df.drop(['faves', 'unfaves'], axis = 1, inplace = True)

In [12]:
t2_df.columns

Index(['UserId', 'AdId', 'Age', 'Cap/Zip-Code', 'Countries visited',
       'Fave Sports', 'Gender', 'Home country', 'Home town', 'Income',
       'Last Name', 'Most listened musics', 'Most read books',
       'Most visited websites', 'Most watched movies',
       'Most watched tv programmes', 'Name', 'Paypal', 'Timepass',
       'Type of Job', 'Weekly working hours', 'AdFilePath', 'Rating',
       'faves_embeddings', 'unfaves_embeddings'],
      dtype='object')

In [13]:
unnecessary_columns = [
    'UserId', 
    'Cap/Zip-Code',
    'Name', 
    'Paypal',
    'AdFilePath'
]
t2_df.drop(unnecessary_columns, axis = 1, inplace = True)

## Ad Tags
Manually made!

In [14]:
tags = pd.read_csv('../../data/tags.csv')
tags.head()

,AdId,accessories,alcohol,animamted,animated,antique,attractive,baby,banner,bar,...,used,wallet,wallets,warranty,white background,white-background,wholesale,wings,woman,work
0,A01_01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A01_02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A01_03,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
3,A01_04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A01_05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
t2_df['AdId']

0        A01_01
1        A01_02
2        A01_03
3        A01_04
4        A01_05
          ...  
36115    A20_11
36116    A20_12
36117    A20_13
36118    A20_14
36119    A20_15
Name: AdId, Length: 36120, dtype: object

In [16]:
tags['AdId']

0      A01_01
1      A01_02
2      A01_03
3      A01_04
4      A01_05
        ...  
296    A20_11
297    A20_12
298    A20_13
299    A20_14
300    A20_15
Name: AdId, Length: 301, dtype: object

In [17]:
full_df = pd.concat([t2_df, tags], axis = 1)

In [18]:
full_df.drop(['AdId'], axis = 1, inplace = True)

## Categorical features
One Hot Encoder for all that aren't multi-label.

In [19]:
categorical_features = [
    'Gender',
    'Home country',
    'Home town',
    'Timepass',
    'Type of Job',
    'Weekly working hours'
]

In [20]:
full_df_dummies = pd.get_dummies(full_df, columns = categorical_features)

In [21]:
full_df_dummies.columns

Index(['Age', 'Countries visited', 'Fave Sports', 'Income', 'Last Name',
       'Most listened musics', 'Most read books', 'Most visited websites',
       'Most watched movies', 'Most watched tv programmes',
       ...
       'Type of Job_Consulting', 'Type of Job_Contract employment',
       'Type of Job_Housewife/Househusband', 'Type of Job_Odd job',
       'Type of Job_Self-employment', 'Type of Job_Student',
       'Type of Job_Temporary', 'Type of Job_Unemploied',
       'Weekly working hours_Full Time', 'Weekly working hours_Part Time'],
      dtype='object', length=283)

### Multilabel

In [101]:
def multilabel_binarizer(df_col):
    unique_classes = []
    for line in range(0, len(df_col)):
        for item in range(0, len(df_col[line].split(','))):
            unique_classes.append(df_col[line].split(', ')[item])
    binarizer = preprocessing.MultiLabelBinarizer(classes=list(set(unique_classes)))
    binarized_columns = pd.DataFrame(binarizer.fit_transform(df_col), columns = binarizer.classes_)
    return binarized_columns

In [88]:
s = []
for line in range(0, len(full_df_dummies)):
    for item in range(0, len(full_df_dummies['Countries visited'][line].split(','))):
        s.append(full_df_dummies['Countries visited'][line].split(', ')[item])
s

['United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United State

In [100]:
list(set(s))

['Cayman Islands',
 'Russia',
 'Norway',
 'Hungary',
 'Bermuda',
 'French Polynesia',
 'Denmark',
 'Bahamas',
 'Honduras',
 'Republic of Montenegro',
 'Haiti',
 'Nicaragua',
 'United States of America',
 'Ukraine',
 'South Africa',
 'Great Britain',
 'Trinidad &amp; Tobago',
 'Egypt',
 'Cook Islands',
 'Japan',
 'Slovenia',
 'Austria',
 'Germany',
 'Italy',
 'Cuba',
 'Costa Rica',
 'Aruba',
 'Estonia',
 'El Salvador',
 'Guam',
 'Turkey',
 'Greece',
 'Sudan',
 'St Barthelemy',
 'New Zealand',
 'United Arab Emirates',
 'Netherlands (Holland',
 'Sri Lanka',
 'Hawaii',
 'Singapore',
 'Korea South',
 'Dominican Republic',
 'Australia',
 'Mexico',
 'Malaysia',
 'Thailand',
 'India',
 'China',
 'Cyprus',
 'Saudi Arabia',
 'Tahiti',
 'Canada',
 'Moldova',
 'Luxembourg',
 'Antigua &amp; Barbuda',
 'Laos',
 'Europe)',
 'Bahrain',
 'Curacao',
 'Iraq',
 'Brazil',
 'Georgia',
 'Spain',
 'Latvia',
 'Guatemala',
 'Vietnam',
 'Colombia',
 'Netherland Antilles',
 'St Kitts-Nevis',
 'Canary Islands',
 '

In [85]:
full_df['Countries visited'][405].split(',')[1]

' Mexico'

In [102]:
multilabel_columns = [
    'Countries visited',
    'Fave Sports',
    'Most listened musics', 
    'Most read books',
    'Most visited websites',
    'Most watched movies',
    'Most watched tv programmes'
]

In [103]:
for col in multilabel_columns:
    new_cols = multilabel_binarizer(full_df_dummies[col])
    full_df = pd.concat([full_df_dummies, new_cols], axis = 1)

/Users/juliatessler/Repos/mo810/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) [' ', '&', '(', ')', ',', '-', ';', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'P', 'R', 'S', 'T', 'U', 'V', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] will be ignored
  warnings.warn(
/Users/juliatessler/Repos/mo810/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) [' ', '(', ')', ',', '.', 'A', 'B', 'E', 'F', 'G', 'H', 'I', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'W', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y', '\u200e'] will be ignored
  warnings.warn(
/Users/juliatessler/Repos/mo810/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) [' ', '&', '(', ')', ',', '-', '.', '

In [96]:
full_df_dummies['Countries visited'].values.tolist()

['United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United State

In [55]:
full_df.columns

Index(['Age', 'Countries visited', 'Fave Sports', 'Income', 'Last Name',
       'Most listened musics', 'Most read books', 'Most visited websites',
       'Most watched movies', 'Most watched tv programmes',
       ...
       'Comedy, Drama, Factual', 'Comedy, Drama, Factual, Learning',
       'Comedy, Factual, Entertainment (Variety Shows)',
       'Comedy, Entertainment (Variety Shows), Music', 'Drama, Comedy',
       'Comedy, Entertainment (Variety Shows), Learning',
       'Comedy, Drama, Learning', 'Children's, Drama, Comedy',
       'Learning, Religion &amp; Ethics, Comedy',
       'Comedy, Drama, Entertainment (Variety Shows), Factual, News'],
      dtype='object', length=359)

## Features

In [ ]:
full_df.columns

In [ ]:
features_df = full_df_dummies.copy()
features_df.drop(
    columns = categorical_features,
    axis = 1,
    inplace = True
)
features_df.drop(
    columns = [
        'UserId', 
        'AdId', 
        'Cap/Zip-Code', 'Countries visited',
       'Fave Sports', 'Gender', 'Home country', 'Home town', 'Income',
       'Last Name', 'Most listened musics', 'Most read books',
       'Most visited websites', 'Most watched movies',
       'Most watched tv programmes', 'Name', 'Paypal', 'Timepass',
       'Type of Job', 'Weekly working hours', 'fave1', 'fave10', 'fave2',
       'fave3', 'fave4', 'fave5', 'fave6', 'fave7', 'fave8', 'fave9',
       'unfave1', 'unfave2', 'unfave3', 'unfave4', 'unfave5', 'unfave6',
       'AdFilePath', 'Rating', 'faves', 'unfaves',
    ],
    axis = 1,
    inplace = True
)

In [ ]:
t2_df['Most read books'].unique()

# Modeling
- plug-in: paramétricos (LDA, KNN, Naive-bayes)
        - LDA e KNN (simples e complexo ou simples, dependendo de K)
- risk minimization: decision trees, nn, etc
    - 

- Falso negativo é pior que falso positivo: causa menos dano mostrar algo que a pessoa não goste tanto do que deixar de mostrar algo que sabemos que a pessoa vai gostar.

- bayesian risk: 